In [10]:
import math
from pymongo import MongoClient
from xml.etree import ElementTree
import pprint
import json 
import matplotlib.pyplot as plt
import matplotlib.path as mplPath
import numpy as np
import urllib.request, json

import operator
import pandas as pd
import numpy as np

In [46]:
start_time = time.time()
conn = MongoClient('120.126.136.17')
db = conn.Tracker
collection = [db.james]# db.db2, db.dn2, db.james, db.leo
userInfo = []
end = datetime.datetime.now().timestamp()
start = (datetime.datetime.now() + datetime.timedelta(days = -14)).timestamp()
for col in collection:
    cursor = col.find({'timestamp': {'$gte': start, '$lt': end}})
    df = pd.DataFrame(list(cursor))
df.replace('', np.nan, inplace=True)
df.fillna(method='ffill', inplace=True)
df['latitude'] = df['latitude'].astype(float).round(4)
df['longitude'] = df['longitude'].astype(float).round(4)
print("--- %s seconds ---" % (time.time() - start_time))

--- 13.788395881652832 seconds ---


In [47]:
df.head()

,_id,day,hour,hr_value,latitude,longitude,minute,month,o2_value,second,step_value,timestamp,year
0,5cc1aef8c4e5a668b7efcca4,25,20,80,24.9522,121.375,58,4,97,9,0,1.556197e+09,2019
1,5cc1af38c4e5a668b7efcca6,25,20,80,24.9522,121.375,59,4,97,13,0,1.556197e+09,2019
2,5cc1af53c4e5a668b7efcca8,25,20,87,24.9522,121.375,59,4,97,40,0,1.556197e+09,2019
3,5cc1af6cc4e5a668b7efccaa,25,21,87,24.9522,121.375,0,4,97,5,0,1.556197e+09,2019
4,5cc1af89c4e5a668b7efccac,25,21,87,24.9522,121.375,0,4,97,35,0,1.556197e+09,2019


In [49]:
lattestData = df[-1:]
baseLocation = [lattestData['latitude'], lattestData['longitude']]
downlat = float(baseLocation[0]) - 0.01#24.938590 
leftlong = float(baseLocation[1]) - 0.01#121.360761

24.9478 121.3745


In [56]:
squarefreq = df.groupby(df[['longitude','latitude']].columns.tolist(),as_index=False).size()

In [62]:
squarefreq

(121.3519, 24.9546)